# Instalar librerias

In [1]:
# !pip install opencv-python
# !pip install numpy tensorflow mediapipe paho-mqtt

# Importar librerias



In [6]:
import cv2
import numpy as np
import math, time
import tensorflow as tf
from tensorflow.keras.models import load_model
import mediapipe as mp
import paho.mqtt.client as paho
from paho import mqtt
import csv
import json

In [7]:
# setting callbacks for different events to see if it works, print the message etc.
def on_connect(client, userdata, flags, rc, properties=None):
    print("CONNACK received with code %s." % rc)

# with this callback you can see if your publish was successful
def on_publish(client, userdata, mid, properties=None):
    print("client: " + str(client) + "userdata: " + str(userdata)+"mid: " + str(mid))

# print which topic was subscribed to
def on_subscribe(client, userdata, mid, granted_qos, properties=None):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))

# Initialize a dictionary to store received messages with timestamps as keys
received_messages = {}

# Function to save the received messages to a CSV file
def save_to_csv(data, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Temperature", "Humidity"])  # Write the header row
        for date, values in data.items():
            writer.writerow([date, values["temperature"], values["humidity"]])

# Function to load existing data from a CSV file
def load_from_csv(filename):
    data = {}
    try:
        with open(filename, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                date = row["Date"]
                temperature = row["Temperature"]
                humidity = row["Humidity"]
                data[date] = {"temperature": temperature, "humidity": humidity}
    except FileNotFoundError:
        pass  # File doesn't exist initially, that's okay
    return data

# Callback to handle incoming messages
def on_message(client, userdata, msg):
    try:
        payload = msg.payload.decode('utf-8')  # Decode the bytes to a string
        data = json.loads(payload)  # Parse the JSON data
        timestamp = data['date']

        # Load existing data from the CSV file
        received_messages = load_from_csv("received_data.csv")

        if timestamp not in received_messages:
            print(f"Received message on topic: {msg.topic}")
            print(f"Date: {data['date']}, Temperature: {data['temperature']}, Humidity: {data['humidity']}")
            received_messages[timestamp] = {
                "temperature": data['temperature'],
                "humidity": data['humidity']
            }

            # Save the received messages to a CSV file
            save_to_csv(received_messages, "received_data.csv")

    except json.JSONDecodeError:
        print("Received payload is not a valid JSON.")

    except KeyError as e:
        print(f"KeyError: {str(e)}. Please check the structure of the received data.")


In [8]:
# Modelo CCN
model = load_model(f'mp_mano_gestos')

# Clases
with open('gestos.names', 'r') as f:
    classNames = f.read().split('\n')

# Initializar Mediapipe
mp_dibujo = mp.solutions.drawing_utils
mp_manos = mp.solutions.hands

# Initializar la camara
cap = cv2.VideoCapture(0)

# Variables
enviar_comando = False
command = ""
ret  = 'none'
sent = False
gesture_sequence = []
gesture_sequence_max_length = 2

In [9]:
# MQTT
ADDR     = "broker.hivemq.com"

# using MQTT version 5 here, for 3.1.1: MQTTv311, 3.1: MQTTv31
# userdata is user defined data of any type, updated by user_data_set()
# client_id is the given name of the client
client = paho.Client(client_id="", userdata=None, protocol=paho.MQTTv5)
client.on_connect = on_connect

# connect to HiveMQ Cloud on port 8883 (default for MQTT)
client.connect(ADDR, 1883)

# setting callbacks, use separate functions like above for better visibility
client.on_subscribe = on_subscribe
client.on_message = on_message
client.on_publish = on_publish

# subscribe to all topics of encyclopedia by using the wildcard "#"
client.subscribe("pjriosc/seminario/IoT/sensors", qos=1)

(0, 1)

In [10]:
client.loop_start()
# Iniciando bucle de captura de video
with mp_manos.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Volteando horizontalmente para visualización natural
        image = cv2.flip(image, 1)

        # Configurando la bandera de escritura
        image.flags.writeable = False

        # Detecciones de mano
        results = hands.process(image)

        # Configurando la bandera de escritura nuevamente
        image.flags.writeable = True

        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Contando los dedos
        finger_count = 0
        gesture = ''  # Inicializando la variable de gesto

        # Renderizando los resultados
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:

                # Mediciones de distancias
                thumb_tip = hand_landmarks.landmark[mp_manos.HandLandmark.THUMB_TIP]
                thumb_ip = hand_landmarks.landmark[mp_manos.HandLandmark.THUMB_IP]
                pinky_mcp = hand_landmarks.landmark[mp_manos.HandLandmark.PINKY_MCP]

                thumb_tip_to_pinky_mcp = math.dist((thumb_tip.x, thumb_tip.y), (pinky_mcp.x, pinky_mcp.y))
                thumb_ip_to_pinky_mcp = math.dist((thumb_ip.x, thumb_ip.y), (pinky_mcp.x, pinky_mcp.y))

                # Pulgar
                if thumb_tip_to_pinky_mcp > thumb_ip_to_pinky_mcp:
                    finger_count += 1

                # Dedo índice
                if hand_landmarks.landmark[mp_manos.HandLandmark.INDEX_FINGER_TIP].y < hand_landmarks.landmark[mp_manos.HandLandmark.INDEX_FINGER_PIP].y:
                    finger_count += 1

                # Dedo medio
                if hand_landmarks.landmark[mp_manos.HandLandmark.MIDDLE_FINGER_TIP].y < hand_landmarks.landmark[mp_manos.HandLandmark.MIDDLE_FINGER_PIP].y:
                    finger_count += 1

                # Dedo anular
                if hand_landmarks.landmark[mp_manos.HandLandmark.RING_FINGER_TIP].y < hand_landmarks.landmark[mp_manos.HandLandmark.RING_FINGER_PIP].y:
                    finger_count += 1

                # Dedo meñique
                if hand_landmarks.landmark[mp_manos.HandLandmark.PINKY_TIP].y < hand_landmarks.landmark[mp_manos.HandLandmark.PINKY_PIP].y:
                    finger_count += 1

                mp_dibujo.draw_landmarks(image, hand_landmarks, mp_manos.HAND_CONNECTIONS,
                                        mp_dibujo.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_dibujo.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )

                # Predicción de gestos
                landmarks = np.array([[int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])] for lm in hand_landmarks.landmark])
                landmarks = np.expand_dims(landmarks, axis=0)
                prediction = model.predict(landmarks)
                classID = np.argmax(prediction)
                gesture = classNames[classID]

        if gesture == 'thumbs down' and enviar_comando == True:
            enviar_comando = False
            command = 'stop'
            client.publish('pjriosc/seminario/IoT', command, retain=True, qos=1)  # Publish MQTT command
            sent = False

        if gesture == 'thumbs up' and enviar_comando == False:
            enviar_comando = True
            command = 'start'
            client.publish('pjriosc/seminario/IoT', command, retain=True, qos=1)  # Publish MQTT command

        if enviar_comando == True and gesture == "peace":
            # Enviar un comando para encender luces
            enviar_comando = False
            command = 'display'
            client.publish('pjriosc/seminario/IoT', command, retain=True, qos=1)  # Publish MQTT command
            time.sleep(0.01)
            
        if gesture == 'stop' and enviar_comando:
            gesture_sequence.append('palm')

        if gesture == 'fist' and 'palm' in gesture_sequence:
            gesture_sequence.append('fist')

        if len(gesture_sequence) > gesture_sequence_max_length:
            gesture_sequence.pop(0)

        # Check if the gesture sequence is 'palm' followed by 'fist'
        if gesture_sequence == ['palm', 'fist']:
            gesture_sequence = []  # Reset the gesture sequence
            command = 'collect'
            client.publish('pjriosc/seminario/IoT', command, retain=True, qos=1)  # Publish MQTT command
            sent = True
            enviar_comando = False

        # Mostrar el conteo de dedos y la predicción de gesto
        # Define text properties
        text_font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = .60
        font_thickness = 2
        text_color = (255, 255, 255)
        text_spacing = 30

        # Prepare a single line text message
        info_text = f'CD: {finger_count} | G: {gesture} | E: {enviar_comando}| CMD: {command}'

        # Calculate text position
        text_y = 20  # Starting position

        # Render the single line text message
        cv2.putText(image, info_text, (10, text_y), text_font, font_scale, text_color, font_thickness, cv2.LINE_AA)


                # Mostrar el fotograma de salida final
        cv2.imshow('Seguimiento de Mano y Reconocimiento de Gesto', image)

        # Romper el bucle si se presiona la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            command = 'idle'
            client.publish('pjriosc/seminario/IoT', command, retain=True, qos=1)  # Publish MQTT command
            break

# Liberar la cámara web y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()
client.loop_stop()

CONNACK received with code Success.
Subscribed: 1 [<paho.mqtt.reasoncodes.ReasonCodes object at 0x000002C705441E10>]
1/1 [==============================] - 0s 20ms/step
client: <paho.mqtt.client.Client object at 0x000002C706FEF190>userdata: Nonemid: 2
1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 20ms/step
client: <paho.mqtt.client.Client object at 0x000002C706FEF190>userdata: Nonemid: 3
1/1 [==============================] - 0s 24ms/step
Received message on topic: pjriosc/seminario/IoT/sensors
Date: 2023-10-21 17:24:35, Temperature: 24, Humidity: 40
1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 28ms/step
client: <paho.mqtt.client.Client object at 0x000002C706FEF190>userdata: Nonemid: 4


In [81]:
client.disconnect()

0